If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it. We also use the `sacrebleu` and `sentencepiece` libraries - you may need to install these even if you already have 🤗 Transformers!

In [ ]:
! pip install transformers[sentencepiece] datasets
! pip install sacrebleu sentencepiece
! pip install huggingface_hub

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 8.9 MB/s eta 0:00:00


In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.1 MB/s eta 0:00:00


If you're opening this notebook locally, make sure your environment has an install from the last version of those libraries.

To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow.

First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then uncomment the following cell and input your token:

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


Make sure your version of Transformers is at least 4.16.0 since some of the functionality we use was introduced in that version:

In [ ]:
import transformers

print(transformers.__version__)

4.44.2


You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/main/examples/tensorflow/translation).

We also quickly upload some telemetry - this tells us which examples and software versions are getting used so we know where to prioritize our maintenance efforts. We don't collect (or care about) any personally identifiable information, but if you'd prefer not to be counted, feel free to skip this step or delete this cell entirely.

In [ ]:
from transformers.utils import send_example_telemetry

send_example_telemetry("translation_notebook", framework="tensorflow")

# Fine-tuning a model on a translation task

In this notebook, we will see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model for a translation task. We will use the [WMT dataset](http://www.statmt.org/wmt16/), a machine translation dataset composed from a collection of various sources, including news commentaries and parliament proceedings.

![Widget inference on a translation task](https://github.com/huggingface/notebooks/blob/main/examples/images/translation.png?raw=1)

We will see how to easily load the dataset for this task using 🤗 Datasets and how to fine-tune a model on it using Keras.

In [ ]:
model_checkpoint = "google/mt5-base"

This notebook is built to run  with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a sequence-to-sequence version in the Transformers library. Here we picked the [`Helsinki-NLP/opus-mt-en-romance`](https://huggingface.co/Helsinki-NLP/opus-mt-en-ROMANCE) checkpoint.

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the `datasets` function `load_dataset` and the `evaluate` function `load`. We use the English/Romanian part of the WMT dataset here.

In [ ]:
file_name = '/data/ctfl_cleaned_train.csv'

In [ ]:
import pandas as pd
def select_columns(input_file, output_file, columns):
    # Read the input CSV file into a pandas DataFrame
    df = pd.read_csv(input_file)

    # Select the specified columns
    selected_columns_df = df[columns]

    # Save the selected columns to a new CSV file
    selected_columns_df.to_csv(output_file, index=False)

# Example usage
input_file = file_name
output_file = 'output.csv'
columns_to_select = ['text', 'expected']

select_columns(input_file, output_file, columns_to_select)

In [ ]:
##Concatenate with the synthetic dataset
import csv

# File paths
file1_path = 'output.csv'
file2_path = #selected file path for the synthetic dataset
#'/data/back_translation7021.csv'
#'data/generated_instruction - generated_instruction.csv'
output_path = 'concatenated_file.csv'

# Read data from the first CSV file
data_from_file1 = []
with open(file1_path, 'r', newline='') as file1:
    reader = csv.reader(file1)
    for row in reader:
        data_from_file1.append(row)

# Read data from the second CSV file, skipping the first row
data_from_file2 = []
max_lines = 7018 #10004
with open(file2_path, 'r', newline='') as file2:
    reader = csv.reader(file2)
    next(reader)  # Skip the first row
    for i, row in enumerate(reader):
        if i >= max_lines:
            break
        data_from_file2.append(row)

# Concatenate data from both files
concatenated_data = data_from_file1 + data_from_file2

# Write the concatenated data to a new CSV file
with open(output_path, 'w', newline='') as outfile:
    writer = csv.writer(outfile)
    writer.writerows(concatenated_data)

print("CSV files concatenated successfully.")


CSV files concatenated successfully.


In [ ]:
from datasets import load_dataset
from evaluate import load

raw_datasets = load_dataset("csv", data_files= 'concatenated_file.csv')
metric = load("sacrebleu")

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set:

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'expected'],
        num_rows: 10660
    })
})

To access an actual element, you need to select a split first, then give an index:

In [ ]:
raw_datasets["train"][0]

{'text': 'พวกเราไม่ค่อยโกรดไคร ยกเว้นคนที่โกหกบ่อย ๆ',
 'expected': 'พวกเราไม่ค่อยโกรธใคร ยกเว้นคนที่โกหกบ่อย ๆ'}

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML


def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(
        dataset
    ), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset) - 1)
        while pick in picks:
            pick = random.randint(0, len(dataset) - 1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(raw_datasets["train"])

,text,expected
0,คุณไม่สบายเหรอครับ,คุณไม่สบายเหรอครับ
1,คุณแน่ใจเกี่ยวกับเรื่องนั้นหรือไม่? พวกเขาไม่ได้ปรุงมันคุณรู้ไหม,แน่ใจนะคะ? เขาไม่อบให้นะคะ?
2,นอกจากนี้ สถานเอกอัครราชทูตฝรั่งเศสประจำประเทศไทย ร่วมกับ Alliance Française Bangkok มีแผนที่จะจัดกิจกรรมทางวัฒนธรรมต่างๆ เพื่อเฉลิมฉลองโอกาสสำคัญนี้ ได้แก่ คอนเสิร์ตดนตรีสไตล์บาโรก การฉายภาพยนตร์ และการสัมมนาในธีมพระราชวังแวร์ซาย การแสดงร่วมสมัย ศิลปะและอาหารฝรั่งเศส,การร่วมนำเสนอรายงาน VNR ของคณะผู้แทนไทย ซึ่งเป็นไปโดยสมัครใจ เป็นการยืนยันเจตนารมณ์ของไทยในการส่งเสริมการดำเนินการตามเป้าหมายการพัฒนาที่ยั่งยืน ซึ่งไทยยังมีโอกาสได้นำเสนอแนวทางการพัฒนาตามหลักปรัชญาของเศรษฐกิจพอเพียงในฐานะแนวทางหนึ่งเพื่อบรรลุวาระการพัฒนาที่ยั่งยืน ทั้งนี้ มีประเทศต่าง ๆ นำเสนอรายงาน VNR ในครั้งนี้จำนวน 44 ประเทศ
3,กลุ่มมนุษยธรรมเร่งเร่งเสริมสร้างที่พักพิงขั้นพื้นฐานที่ชาวโรฮิงญาสร้างขึ้นอย่างเร่งรีบ ขณะที่พวกเขาหลบหนีข้ามพรมแดน หลังจากการปราบปรามของกองทัพอย่างรุนแรงต่อชุมชนทางตะวันตกของพม่า,กลุ่มความช่วยเหลือด้านมนุษยธรรมได้เร่งเพิ่มความแข็งแรงให้กับที่หลบภัยที่ชาวโรฮีนจาได้สร้างขึ้นอย่างรีบร้อนขณะหนีข้ามชายแดนจากการปราบปรามด้วยกองทัพอย่างโหดเหี้ยมในพื้นที่ชุมชนเขตตะวันตกของพม่า
4,ในภาคกลางมีกรุงโซลแล้วก็เกาหลีเหนือทั้งหมดแบ่งออกเป็นภาคเหนือ,ในภาคกลางมีกรุงโซล และเกาหลีเหนือทั้งหมดจัดเป็นภาคเหนือ


The metric is an instance of [`datasets.Metric`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Metric):

In [ ]:
metric

EvaluationModule(name: "sacrebleu", module_type: "metric", features: [{'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id='references')}, {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}], usage: """
Produces BLEU scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions (`list` of `str`): list of translations to score. Each translation should be tokenized into a list of tokens.
    references (`list` of `list` of `str`): A list of lists of references. The contents of the first sub-list are the references for the first prediction, the contents of the second sub-list are for the second prediction, etc. Note that there must be the same number of references for each prediction (i.e. all sub-lists must be of the same length).
    smooth_method (`str`): The smoothing method to use, defaults to `'e

You can call its `compute` method with your predictions and labels, which need to be list of decoded strings (list of list for the labels):

In [ ]:
fake_preds = ["hello there", "general kenobi"]
fake_labels = [["hello there"], ["general kenobi"]]
metric.compute(predictions=fake_preds, references=fake_labels)

{'score': 0.0,
 'counts': [4, 2, 0, 0],
 'totals': [4, 2, 0, 0],
 'precisions': [100.0, 100.0, 0.0, 0.0],
 'bp': 1.0,
 'sys_len': 4,
 'ref_len': 4}

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/376 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting

By default, the call above will use one of the fast tokenizers (backed by Rust) from the 🤗 Tokenizers library.

You can directly call this tokenizer on one sentence or a pair of sentences:

In [ ]:
tokenizer("Hello, this is a sentence!")

{'input_ids': [30273, 261, 714, 339, 259, 262, 259, 98923, 309, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

Instead of one sentence, we can pass along a list of sentences:

In [ ]:
tokenizer(["Hello, this is a sentence!", "This is another sentence."])

{'input_ids': [[30273, 261, 714, 339, 259, 262, 259, 98923, 309, 1], [1494, 339, 259, 7845, 259, 98923, 260, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]]}

To prepare the targets for our model, we need to tokenize them inside the `as_target_tokenizer` context manager. This will make sure the tokenizer uses the special tokens corresponding to the targets:

In [ ]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["Hello, this is a sentence!", "This is another sentence."]))

{'input_ids': [[30273, 261, 714, 339, 259, 262, 259, 98923, 309, 1], [1494, 339, 259, 7845, 259, 98923, 260, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]]}


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


If you are using one of the five T5 checkpoints that require a special prefix to put before the inputs, you should adapt the following cell.

In [ ]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "edit: "
else:
    prefix = ""

We can then write the function that will preprocess our samples. We just feed them to the `tokenizer` with the argument `truncation=True`. This will ensure that an input longer that what the model selected can handle will be truncated to the maximum length accepted by the model. The padding will be dealt with later on (in a data collator) so we pad examples to the longest length in the batch and not the whole dataset.

In [ ]:
max_input_length = 1024
max_target_length = 128


def preprocess_function(examples):
    inputs = [prefix for ex in examples["text"]]
    targets = [ex for ex in examples["expected"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [ ]:
preprocess_function(raw_datasets["train"][:2])

{'input_ids': [[1], [1]], 'attention_mask': [[1], [1]], 'labels': [[259, 90297, 136732, 43957, 10228, 27987, 35468, 259, 235077, 84494, 43957, 211411, 159218, 259, 4571, 1], [51738, 65194, 818, 259, 87876, 259, 65211, 65194, 53959, 97354, 62883, 57359, 23941, 222953, 5405, 43957, 113962, 6196, 1]]}

To apply this function on all the pairs of sentences in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/10660 [00:00<?, ? examples/s]

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since our task is of the sequence-to-sequence kind, we use the `AutoModelForSeq2SeqLM` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us.

In [ ]:
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq

model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

tf_model.h5:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/tf_keras/src/initializers/initializers.py:121: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
All model checkpoint layers were used when initializing TFMT5ForConditionalGeneration.

All the layers of TFMT5ForConditionalGeneration were initialized from the model checkpoint at google/mt5-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMT5ForConditionalGeneration for predictions without further training.


generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Note that  we don't get a warning like in our classification example. This means we used all the weights of the pretrained model and there is no randomly initialized head in this case.

In [ ]:
batch_size = 16
learning_rate = 2e-5
weight_decay = 0.01
num_train_epochs = 30

model_name = model_checkpoint.split("/")[-1]
push_to_hub_model_id = #model_id

Then, we need a special kind of data collator, which will not only pad the inputs to the maximum length in the batch, but also the labels. Note that our data collators are designed to work for multiple frameworks, so ensure you set the `return_tensors='np'` argument to get NumPy arrays out - you don't want to accidentally get a load of `torch.Tensor` objects in the middle of your nice TF code! You could also use `return_tensors='tf'` to get TensorFlow tensors, but our TF dataset pipeline actually uses a NumPy loader internally, which is wrapped at the end with a `tf.data.Dataset`. As a result, `np` is usually more reliable and performant when you're using it!

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="np")

generation_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="np", pad_to_multiple_of=128)

Next, we convert our datasets to `tf.data.Dataset`, which Keras understands natively. There are two ways to do this - we can use the slightly more low-level [`Dataset.to_tf_dataset()`](https://huggingface.co/docs/datasets/package_reference/main_classes#datasets.Dataset.to_tf_dataset) method, or we can use [`Model.prepare_tf_dataset()`](https://huggingface.co/docs/transformers/main_classes/model#transformers.TFPreTrainedModel.prepare_tf_dataset). The main difference between these two is that the `Model` method can inspect the model to determine which column names it can use as input, which means you don't need to specify them yourself. Make sure to specify the collator we just created as our `collate_fn`!

We also want to compute `BLEU` metrics, which will require us to generate text from our model. To speed things up, we can compile our generation loop with XLA. This results in a *huge* speedup - up to 100X! The downside of XLA generation, though, is that it doesn't like variable input shapes, because it needs to run a new compilation for each new input shape! To compensate for that, let's use `pad_to_multiple_of` for the dataset we use for text generation. This will reduce the number of unique input shapes a lot, meaning we can get the benefits of XLA generation with only a few compilations.

In [ ]:
train_dataset = model.prepare_tf_dataset(
    tokenized_datasets["train"],
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator,
)

validation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["train"],
    batch_size=batch_size,
    shuffle=False,
    collate_fn=data_collator,
)

generation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["train"],
    batch_size=8,
    shuffle=False,
    collate_fn=generation_data_collator,
)

Now we initialize our loss and optimizer and compile the model. Note that most Transformers models compute loss internally, so we can just leave the loss argument blank to use the internal loss instead. For the optimizer, we can use the `AdamWeightDecay` optimizer in the Transformer library.

In [ ]:
from transformers import AdamWeightDecay
import tensorflow as tf

optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
model.compile(optimizer=optimizer)

Now we can train our model. We can also add a few optional callbacks here, which you can remove if they aren't useful to you. In no particular order, these are:
- PushToHubCallback will sync up our model with the Hub - this allows us to resume training from other machines, share the model after training is finished, and even test the model's inference quality midway through training!
- TensorBoard is a built-in Keras callback that logs TensorBoard metrics.
- KerasMetricCallback is a callback for computing advanced metrics. There are a number of common metrics in NLP like ROUGE which are hard to fit into your compiled training loop because they depend on decoding predictions and labels back to strings with the tokenizer, and calling arbitrary Python functions to compute the metric. The KerasMetricCallback will wrap a metric function, outputting metrics as training progresses.

If this is the first time you've seen `KerasMetricCallback`, it's worth explaining what exactly is going on here. The callback takes two main arguments - a `metric_fn` and an `eval_dataset`. It then iterates over the `eval_dataset` and collects the model's outputs for each sample, before passing the `list` of predictions and the associated `list` of labels to the user-defined `metric_fn`. If the `predict_with_generate` argument is `True`, then it will call `model.generate()` for each input sample instead of `model.predict()` - this is useful for metrics that expect generated text from the model, like `ROUGE` and `BLEU`.

This callback allows complex metrics to be computed each epoch that would not function as a standard Keras Metric. Metric values are printed each epoch, and can be used by other callbacks like `TensorBoard` or `EarlyStopping`.

In [ ]:
from transformers.keras_callbacks import KerasMetricCallback
import numpy as np


def metric_fn(eval_predictions):
    preds, labels = eval_predictions
    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds
    ]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # We use -100 to mask labels - replace it with the tokenizer pad token when decoding
    # so that no output is emitted for these
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}
    result["gen_len"] = np.mean(prediction_lens)
    return result


metric_callback = KerasMetricCallback(
    metric_fn=metric_fn, eval_dataset=generation_dataset, predict_with_generate=True, use_xla_generation=True,
    generate_kwargs={"max_length": 128}
)

With the metric callback ready, now we can specify the other callbacks and fit our model:

In [ ]:
from transformers.keras_callbacks import PushToHubCallback
#from tensorflow.keras.callbacks import TensorBoard

#tensorboard_callback = TensorBoard(log_dir="./translation_model_save/logs")

push_to_hub_callback = PushToHubCallback(
    output_dir="./translation_model_save",
    tokenizer=tokenizer,
    hub_model_id=push_to_hub_model_id,
)

callbacks = [metric_callback, push_to_hub_callback]

model.fit(
    train_dataset, validation_data=validation_dataset, epochs=30, callbacks=callbacks
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/pakawadeep/mt5-base-finetuned-ctfl-backtranslation_7k into local empty directory.


Download file tf_model.h5:   0%|          | 7.55k/3.60G [00:00<?, ?B/s]

Download file tokenizer.json:   0%|          | 8.30k/15.6M [00:00<?, ?B/s]

Download file spiece.model:   1%|          | 32.0k/4.11M [00:00<?, ?B/s]

Clean file spiece.model:   0%|          | 1.00k/4.11M [00:00<?, ?B/s]

Clean file tokenizer.json:   0%|          | 1.00k/15.6M [00:00<?, ?B/s]

Clean file tf_model.h5:   0%|          | 1.00k/3.60G [00:00<?, ?B/s]

Epoch 1/30
666/666 [==============================] - 794s 1s/step - loss: 16.3214 - val_loss: 7.9429 - bleu: 8.4265e-04 - gen_len: 127.0000
Epoch 2/30
666/666 [==============================] - 689s 1s/step - loss: 9.3082 - val_loss: 7.3301 - bleu: 0.0000e+00 - gen_len: 127.0000
Epoch 3/30
666/666 [==============================] - 689s 1s/step - loss: 8.1593 - val_loss: 7.1451 - bleu: 0.0000e+00 - gen_len: 127.0000
Epoch 4/30
666/666 [==============================] - 333s 501ms/step - loss: 7.7440 - val_loss: 7.0581 - bleu: 0.0000e+00 - gen_len: 8.0000
Epoch 5/30
666/666 [==============================] - 331s 497ms/step - loss: 7.5624 - val_loss: 7.0120 - bleu: 0.0000e+00 - gen_len: 3.0000
Epoch 6/30
666/666 [==============================] - 324s 487ms/step - loss: 7.4569 - val_loss: 6.9663 - bleu: 0.0000e+00 - gen_len: 3.0000
Epoch 7/30
666/666 [==============================] - 651s 979ms/step - loss: 7.3873 - val_loss: 6.9154 - bleu: 9.9007e-05 - gen_len: 114.0000
Epoch 8/30
66

OSError: Error cleaning LFS object: write /content/translation_model_save/.git/lfs/tmp/2214012645: no space left on device
error: packet write failed: Broken pipe
error: external filter 'git-lfs filter-process' failed
fatal: tf_model.h5: clean filter 'lfs' failed


If you used the callback above, you can now share this model with all your friends, family or favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import TFAutoModelForSeq2SeqLM

model = TFAutoModelForSeq2SeqLM.from_pretrained("your-username/my-awesome-model")
```

## Inference

Now we've trained our model, let's see how we could load it and use it to translate text in future! First, let's load it from the hub. This means we can resume the code from here without needing to rerun everything above every time.

In [ ]:
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM

# You can of course substitute your own username and model here if you've trained and uploaded it!
model_name = #model_name
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


tf_model.h5:  34%|###3      | 2.34G/6.97G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/tf_keras/src/initializers/initializers.py:121: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
All model checkpoint layers were used when initializing TFMT5ForConditionalGeneration.

All the layers of TFMT5ForConditionalGeneration were initialized from the model checkpoint at pakawadeep/mt5-large-finetuned-ctfl-augmented_2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMT5ForConditionalGeneration for predictions without further training.


generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

Now let's try tokenizing some text and passing it to the model to generate a translation. Don't forget to add the "translate: " string at the start if you're using a `T5` model.

In [ ]:
input_text  = "เขากินม้อเช้าเร็วเพราะจะทำงานไม่ทัน"
if 't5' in model_name:
    input_text = "edit: " + input_text
tokenized = tokenizer([input_text], return_tensors='np')
out = model.generate(**tokenized, max_length=128)
with tokenizer.as_target_tokenizer():
    print(tokenizer.decode(out[0], skip_special_tokens=True))

edit: เขากินมื้อเช้าเร็วเพราะจะทํางานไม่ทัน


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Well, that's some tokens and a lot of padding! Let's decode those to see what it says, using the `skip_special_tokens` argument to skip those padding tokens:

In [ ]:
input_text  = "แมวกินปลาเป็นอหารเชา"
if 't5' in model_name:
    input_text = "edit: " + input_text
tokenized = tokenizer([input_text], return_tensors='np')
out = model.generate(**tokenized, max_length=128)
with tokenizer.as_target_tokenizer():
    print(tokenizer.decode(out[0], skip_special_tokens=True))

แก้ไข: แมวกินปลาเป็นอหารเชา


##Run on the Test Set for Evaluation##

In [ ]:
file_test = '/data/ctfl_cleaned_test.csv'

In [ ]:
import csv

def read_column_as_list(file_path, column_index):
    column_data = []
    with open(file_path, 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            if len(row) > column_index:
                column_data.append(row[column_index])
    return column_data

# Replace 'file_path.csv' with the path to your CSV file
column_index = 2  # Index 2 corresponds to the third column (0-indexed)

column_as_list = read_column_as_list(file_test, column_index)
test_list = column_as_list[1:]


In [ ]:
test_list[0]

'ออกกำลังกายบ่อยๆทำให้สุขภาพดี'

In [ ]:
csv_file_path = #saved directory

with open(csv_file_path, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['text', 'expected'])  # Writing header

In [ ]:
with open(csv_file_path, 'w', newline='') as file:
  writer = csv.writer(file)
  for input_text in test_list:
    input_text = "edit: " +  input_text
    tokenized = tokenizer([input_text], return_tensors='np')
    out = model.generate(**tokenized, max_length=128)
    with tokenizer.as_target_tokenizer():
      writer.writerow([input_text, tokenizer.decode(out[0], skip_special_tokens=True)])
    # print(tokenizer.decode(out[0]))

This is the point where I start wishing I'd done this example in a language I actually speak. Still, it looks good! Probably!